<a href="https://colab.research.google.com/github/CameronKenworthyCode/NEWICC/blob/main/region_fires_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [217]:
#initialize notebook
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re

!pip install pykml
from pykml.factory import KML_ElementMaker as KML

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1NI4dy7k8xTTMlLlrZFIRPH9sZgIQ4zp1' in parents and trashed=False"}).GetList() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive/


In [218]:
stat_fires = pd.read_excel('drive/My Drive/NEWICC/2022_fires/2021 DNR Firebook.xlsx', sheet_name='DNR Classified')
stat_fires = stat_fires.dropna(axis=0, subset=['Fire Name'])
columns_to_drop = ['Region', 'Fire Number',
       'Other Center WildCAD Inc#', 'Cause',
       'Sub-Cause\nfor MISC cause only', 'Program',
       'DNR Jurisdiction Acres', 'DNR Ownership Acres',
       'Other Agency Jurisdiction Acres', 'Other Agency',
       'Complexity', 'Resource Order Number',
       'Cost Recovery', 'Incident Commander', 'Investigator', 'Pcode',
       'COF/OWF SO#', 'Other Agency Response ', 'Reimbursable ', 'Comments',
       'Estimated Aircraft Costs', 'Estimated Costs', 'New Costs / Courtney',
       'SIT Report  Initials/Date', 'Fire Out Date', 'Fire Out Time',
       'Within 24 Hours/ Assist', 'INFORM', 'IRWIN ID', 'WFIS ', 'NAR', 'WFIR', 'Unnamed: 41']

stat_fires = stat_fires.drop(columns=columns_to_drop)\

stat_fires['Fire Name'] = stat_fires['Fire Name'].apply(str.strip)

false_alarms = pd.read_excel('drive/My Drive/NEWICC/2022_fires/2021 DNR Firebook.xlsx', sheet_name='False Alarm')
false_alarms.columns = ['Fire Number', 'WILDCAD INC', 'Date Reported', 'Fire Name',
       'Reason for False Alarm', 'Dist', 'County', 'Finance Alpha Code',
       'Town', 'Rng', 'Sec']
false_alarms['Fire Name'] = false_alarms['Fire Name'].apply(str.strip)

region_fires =  pd.read_excel('drive/My Drive/NEWICC/2022_fires/region_fires.xlsx', sheet_name='main')

region_fires['Fire Name'] = region_fires['Fire Name'].apply(str.strip)
region_fires_fa = region_fires.query('Classification == "False Alarm"')
region_fires_stat = region_fires.query('Classification == "Classified"')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#sheet headers

In [ ]:
stat_fires

,Finance Alpha Code,WILDCAD INC #,County,Date Reported,Fire Name,Town,Rng,Sec,Dist.,Unnamed: 42
0,AOR,1054.0,STEVENS,2021-01-13,DEER CREEK,30.0,39,2.0,NC,NaN
1,AOS,1061.0,SPOKANE,2021-02-13,DAHL,28.0,42,3.0,AR,NaN
2,AOT,1063.0,STEVENS,2021-02-24,RIDGETOP WAY,35.0,39,2.0,NC,NaN
3,AOU,1081.0,SPOKANE,2021-03-04,WEILE,26.0,43,25.0,AR,NaN
4,AOV,1083.0,SPOKANE,2021-03-04,ORCHARD,26.0,43,24.0,AR,NaN
...,...,...,...,...,...,...,...,...,...,...
784,JLZ,2778.0,SPOKANE,2021-11-17,CHATTAROY DENISON,28.0,43,20.0,AR,NaN
785,JMA,2781.0,SPOKANE,2021-11-18,PINEHILL,24.0,43,21.0,AR,NaN
786,JMB,2784.0,SPOKANE,2021-11-29,KOTH RIDGE,26.0,45,1.0,AR,NaN
787,JMC,2787.0,OKANOGAN,2021-12-01,TALKIRE,36.0,28,4.0,HI,NaN


In [ ]:
false_alarms

,Fire Number,WILDCAD INC,Date Reported,Fire Name,Reason for False Alarm,Dist,County,Finance Alpha Code,Town,Rng,Sec
0,800.0,1055.0,2021-01-13,FALSE MELVILL,"ON IMPROVED PROPERTY, NO DNR",NC,STEVENS,BEI,24.0,41.0,13.0
1,804.0,1090.0,2021-03-10,FALSE SWENSON,"ON IMPROVED PROPERTY, NO DNR",AR,STEVENS,BEI,28.0,41.0,24.0
2,805.0,1092.0,2021-03-11,UPPER SWANSON,"LEGAL PERMIT BURN, NO ESCAPE",HI,OKANOGAN,BEF,39.0,28.0,34.0
3,806.0,1096.0,2021-03-12,FALSE REGAL,"ON IMPROVED PROPERTY, NO DNR",AR,SPOKANE,BEI,29.0,43.0,27.0
4,NaN,1177.0,2021-03-23,CURLEW CEMETARY,"ILLEGAL BURN, NO ESCAPE",HI,FERRY,BEF,39.0,33.0,14.0
5,NaN,1196.0,2021-03-29,LAKEVIEW,"LEGAL RULE BURN, NO ESCAPE",NC,STEVENS,BEH,31.0,40.0,16.0
6,NaN,1304.0,2021-04-15,CAMERON MEADOW,"LEGAL PERMIT BURN, NO ESCAPE",NC,STEVENS,BEH,32.0,37.0,14.0
7,NaN,1341.0,2021-04-17,FALSE HESSLETINE,NO FIRE FOUND,NC,STEVENS,BEH,30.0,40.0,18.0
8,NaN,1462.0,2021-04-23,EMPIRE,"ON IMPROVED PROPERTY, NO DNR",HI,FERRY,BEF,38.0,33.0,4.0
9,NaN,1529.0,2021-05-12,ALM,"ON IMPROVED PROPERTY, NO DNR",NC,STEVENS,BEH,32.0,40.0,23.0


In [ ]:
region_fires

,County,Classification,Fire Number,Start Date,Fire Name,Project Number,Acres Burned,Latitude,Longitude,Complexity
0,Spokane County,False Alarm,1055.0,1/13/2021,FALSE MELVILLE,BEI,NaN,NaN,NaN,NaN
1,Stevens County,Classified,1054.0,1/13/2021,DEER CREEK,AOR,0.1,48.12778,-117.8562,5.0
2,Spokane County,Classified,1061.0,2/13/2021,DAHL,AOS,0.1,47.95980,-117.4905,5.0
3,Stevens County,Classified,1063.0,2/24/2021,RIDGETOP WAY,AOT,21.5,48.55478,-117.8495,4.0
4,Spokane County,Classified,1081.0,3/4/2021,WEILE,AOU,0.1,47.72124,-117.3193,5.0
...,...,...,...,...,...,...,...,...,...,...
1223,Stevens County,False Alarm,NaN,11/30/2021,FA LUTHER,KEI,NaN,48.05747,-117.8291,NaN
1224,Okanogan County,Classified,NaN,12/1/2021,TALKIRE,JMC,2.5,48.64647,-119.3288,5.0
1225,Okanogan County,False Alarm,NaN,12/1/2021,FA PARRY,KEF,NaN,48.70428,-119.4505,NaN
1226,Okanogan County,False Alarm,NaN,12/2/2021,FA ONEIL 97,KEF,NaN,48.81382,-119.4081,NaN


In [ ]:
region_fires_fa

,County,Classification,Fire Number,Start Date,Fire Name,Project Number,Acres Burned,Latitude,Longitude,Complexity
0,Spokane County,False Alarm,1055.0,1/13/2021,FALSE MELVILLE,BEI,NaN,NaN,NaN,NaN
6,Spokane County,False Alarm,1082.0,3/4/2021,FALSE RIVERSIDE,BEI,NaN,47.66444,-117.4641,NaN
7,Spokane County,False Alarm,1084.0,3/5/2021,FALSE MORNINGSIDE,BEI,NaN,47.72256,-117.5503,NaN
8,Stevens County,False Alarm,1085.0,3/5/2021,FALSE ANTLER,BEI,NaN,47.94516,-117.5560,NaN
14,Stevens County,False Alarm,1090.0,3/10/2021,FALSE SWENSON,BEI,NaN,47.91095,-117.5651,NaN
...,...,...,...,...,...,...,...,...,...,...
1219,Lincoln County,False Alarm,NaN,11/18/2021,FA HAWK,KEI,NaN,47.78086,-118.3020,NaN
1222,Stevens County,False Alarm,NaN,11/30/2021,FA COMSTOCK MINE,KEH,NaN,48.67696,-117.8348,NaN
1223,Stevens County,False Alarm,NaN,11/30/2021,FA LUTHER,KEI,NaN,48.05747,-117.8291,NaN
1225,Okanogan County,False Alarm,NaN,12/1/2021,FA PARRY,KEF,NaN,48.70428,-119.4505,NaN


In [ ]:
region_fires_stat

,County,Classification,Fire Number,Start Date,Fire Name,Project Number,Acres Burned,Latitude,Longitude,Complexity
1,Stevens County,Classified,1054.0,1/13/2021,DEER CREEK,AOR,0.1,48.12778,-117.8562,5.0
2,Spokane County,Classified,1061.0,2/13/2021,DAHL,AOS,0.1,47.95980,-117.4905,5.0
3,Stevens County,Classified,1063.0,2/24/2021,RIDGETOP WAY,AOT,21.5,48.55478,-117.8495,4.0
4,Spokane County,Classified,1081.0,3/4/2021,WEILE,AOU,0.1,47.72124,-117.3193,5.0
5,Spokane County,Classified,1083.0,3/4/2021,ORCHARD,AOV,1.0,47.73763,-117.3248,5.0
...,...,...,...,...,...,...,...,...,...,...
1218,Spokane County,Classified,NaN,11/17/2021,CHATTAROY DENISON,JLZ,0.2,47.90628,-117.3927,5.0
1220,Spokane County,Classified,NaN,11/18/2021,PINEHILL,JMA,0.1,47.55910,-117.3831,5.0
1221,Spokane County,Classified,NaN,11/29/2021,KOTH RIDGE,JMB,0.1,47.78455,-117.0603,5.0
1224,Okanogan County,Classified,NaN,12/1/2021,TALKIRE,JMC,2.5,48.64647,-119.3288,5.0


#Missing Values

##region_fires entries missing in firebook

In [ ]:
missing_firebook = []

for i in range(len(region_fires_stat)):
  row = region_fires_stat.iloc[i]
  name = row['Fire Name']
  match = stat_fires[stat_fires['Fire Name'] == name]

  if len(match) == 0:
    missing_firebook.append(region_fires_stat.index[i])



for i in range(len(region_fires_fa)):
  row = region_fires_fa.iloc[i]

  inc_name = row['Fire Name']
  match = false_alarms[false_alarms['Fire Name'] == inc_name]

  if len(match) == 0:
    missing_firebook.append(region_fires_fa.index[i])

missing = region_fires.iloc[missing_firebook].reset_index(drop=True)

missing.to_excel('missing_firebook.xlsx')
!cp missing_firebook.xlsx "drive/My Drive/NEWICC/2022_fires"

missing

,County,Classification,Fire Number,Start Date,Fire Name,Project Number,Acres Burned,Latitude,Longitude,Complexity
0,Grant County,Classified,NaN,4/27/2021,SAND DUNES,LSP,0.1,47.073592,-119.338364,NaN
1,Franklin County,Classified,NaN,5/15/2021,SAGEMOOR,CUM,50.0,46.400550,-119.035400,NaN
2,Benton County,Classified,NaN,6/1/2021,WEBBER,LCU,8.5,46.195930,-119.457600,4.0
3,Grant County,Classified,NaN,6/9/2021,BABCOCK BENCH,CUY,0.5,46.985149,-119.967539,5.0
4,Stevens County,Classified,NaN,6/24/2021,BISSEL FLATS,BCW,0.1,48.183140,-118.158500,5.0
5,Grant County,Classified,NaN,6/30/2021,WANAPUM VILLAGE,CVM,29.0,46.857160,-119.946400,NaN
6,Okanogan County,Classified,NaN,7/4/2021,CAYUSE MOUNTAIN,JAS,0.1,48.676900,-119.222000,5.0
7,Douglas County,Classified,NaN,7/8/2021,WHITEHALL ROAD,LSG,910.0,47.542170,-119.636300,4.0
8,Okanogan County,Classified,NaN,7/12/2021,CHUWEAH CREEK FIRE,KSP,36752.0,48.228610,-119.016900,2.0
9,Grant County,Classified,NaN,7/15/2021,SAND DUNES,LSP,0.3,47.036140,-119.313900,NaN


##firebook entries missing in region_fires

In [219]:
region_missing_stat = []

for i in range(len(stat_fires)):
  row = stat_fires.iloc[i]

  inc_name = row['Fire Name']
  match = region_fires[region_fires['Fire Name'] == inc_name]

  if len(match) == 0:
    region_missing_stat.append(i)

stat_fires.iloc[region_missing_stat]

region_missing_stat = stat_fires.iloc[region_missing_stat]


#region_missing_stat.columns = ['Finance Alpha Code', 'WILDCAD INC #', 'County', 'Date Reported',
#       'Fire Name', 'Town', 'Rng', 'Sec', 'Dist']

region_missing_stat.to_excel('missing_region_fires_stat.xlsx')
!cp missing_region_fires_stat.xlsx "drive/My Drive/NEWICC/2022_fires"

region_missing_stat

,Finance Alpha Code,WILDCAD INC #,County,Date Reported,Fire Name,Town,Rng,Sec,Dist.,Unnamed: 42
369,BCW,1792.0,STEVENS,2021-06-24,BISSELL FLATS,31.0,37,16.0,NC,NaN
422,JAS,1903.0,OKANOGAN,2021-07-04,CAYUSE MOUTAIN,37.0,29,30.0,HI,NaN
475,KSP,2031.0,OKANOGAN,2021-07-12,CHUWEAH CREEK,31.0,31,26.0,SO,NaN
516,JDP,2114.0,OKANOGAN,2001-07-20,NK'MIP,40.0,27,3.0,HI,NaN
590,JFW,2295.0,FERRY,2021-08-04,SEVENTEEN MILE MOUNTAIN,35.0,34,48.0,HI,NaN
624,KTS,2291.0,NaN,2021-08-04,LITTLE ED,35.0,34`,41.0,NC,NaN
640,KTZ,2372.0,KOOTENAI,2021-08-07,IDL STATELINE ROAD,NaN,NaN,NaN,AR,NaN
761,KUV,2712.0,FERRY,2021-10-07,LUPINE,35.0,37,5.0,NC,NaN


In [ ]:
region_missing = stat_fires.iloc[region_missing_stat].drop(columns=['Region', 'Fire Number',
       'Other Center WildCAD Inc#', 'Cause',
       'Sub-Cause\nfor MISC cause only', 'Program',
       'DNR Jurisdiction Acres', 'DNR Ownership Acres',
       'Other Agency Jurisdiction Acres', 'Other Agency',
       'Complexity', 'Resource Order Number',
       'Cost Recovery', 'Incident Commander', 'Investigator', 'Pcode',
       'COF/OWF SO#', 'Other Agency Response ', 'Reimbursable ', 'Comments',
       'Estimated Aircraft Costs', 'Estimated Costs', 'New Costs / Courtney',
       'SIT Report  Initials/Date', 'Fire Out Date', 'Fire Out Time',
       'Within 24 Hours/ Assist', 'INFORM', 'IRWIN ID', 'WFIS ', 'NAR', 'WFIR',
       'Unnamed: 41', 'Unnamed: 42'])

region_missing.columns = ['Finance Alpha Code', 'WILDCAD INC #', 'County', 'Date Reported',
       'Fire Name', 'Town', 'Rng', 'Sec', 'Dist']

In [ ]:
region_missing_fa = []

for i in range(len(false_alarms)):
  row = false_alarms.iloc[i]
  name = row['Fire Name']

  match = region_fires_fa[region_fires['Fire Name'] == name]

  if len(match) == 0:
    region_missing_fa.append(i)

region_missing_fa = false_alarms.iloc[region_missing_fa]

region_missing_fa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,Fire Number,WILDCAD INC,Date Reported,Fire Name,Reason for False Alarm,Dist,County,Finance Alpha Code,Town,Rng,Sec
0,800.0,1055,2021-01-13,FALSE MELVILL,"ON IMPROVED PROPERTY, NO DNR",NC,STEVENS,BEI,24,41,13
20,NaN,1177,2021-03-23,CURLEW CEMETARY,"ILLEGAL BURN, NO ESCAPE",HI,FERRY,BEF,39,33,14
53,NaN,1304,2021-04-15,CAMERON MEADOW,"LEGAL PERMIT BURN, NO ESCAPE",NC,STEVENS,BEH,32,37,14
90,NaN,1462,2021-04-23,EMPIRE,"ON IMPROVED PROPERTY, NO DNR",HI,FERRY,BEF,38,33,4
122,NaN,1600,2021-05-28,FALSE COEUR D'ALENE,OTHER,AR,SPOKANE,BEI,25,42,24
135,NaN,1631,2021-06-02,NO SKIDMORE,"ON IMPROVED PROPERTY, NO DNR",NC,STEVENS,BEH,34,39,9
143,NaN,1667,2021-06-08,FALSE BULLDOG,OTHER,SO,OKANOGAN,BEG,33,26,19
144,NaN,1669,2021-06-08,FALSE CURLEW,"LEGAL PERMIT BURN, NO ESCAPE",HI,FERRY,BEF,37,33,20
180,NaN,1823,2021-06-27,WILLIAMS QUARRY,NO FIRE FOUND,NC,STEVENS,BEH,35,39,4
181,NaN,1821,2021-06-27,FALSE CANYON,NO FIRE FOUND,NC,STEVENS,BEH,32,40,2
